# Prédiction de défaut des prêts SBA

---
# Résumé du Projet

Ce projet, construit sur les prêts historiques du programme **7(a) de la SBA** (US Small Business Administration), vise à prédire le risque de défaut (`LoanStatus`) d'une entreprise participant à ce programme. Ainsi l'objectif du modèle est de distinguer efficacement les prêts remboursés (*Paid in Full*) des pertes sèches (*Charged Off*).

Le projet est construit en plusieurs étapes :

1.  **Analyse Temporelle :** Au début du projet, les données disponibles depuis le début des années 2000 ont été analysées afin de comprendre les relations entre les différentes variables disponibles ainsi que l'évolution dans le temps des défauts. L'analyse a montré la **nécessité** de réduire la période d'étude suite aux variations des cycles macroéconomiques observées dans le temps. Ainsi, la période d'étude a été réduite à **2021-2024** afin de capter les dernières tendances économiques et éviter d'apprendre sur des données qui ne sont aujourd'hui plus d'actualité.

2.  **Enrichissement Macroéconomique :** En plus des variables disponibles dans les tables fournies par la SBA, nous avons ajouté des **variables macroéconomiques** au modèle afin de lui donner des indications sur les périodes de crise et de changement macroéconomique. Afin d'éviter tout *data leakage* possible, mais aussi dans le but de construire un projet réaliste et non seulement prédictif, ces variables macroéconomiques ont été utilisées avec un **décalage temporel** (ex : croissance du PIB de l'année précédente et non de l'année actuelle).

3.  **Validation "Business" :** Afin de capter la pertinence de ce modèle, des règles utilisées directement en entreprise ont été appliquées. Ainsi, le découpage en échantillon *train/test* respecte la temporalité et le modèle final est validé sur un échantillon **Out-of-Time (OOT)** afin de tester la stabilité des performances dans le temps.

4.  **Modélisation Hybride :** Plusieurs modèles seront comparés, en commençant par une **régression logistique** en tant que modèle benchmark, puis des modèles plus avancés (**Boosting, Bagging**) seront également testés. Différentes méthodes de sélection de variables (**Lasso, Elastic Net**) sont évaluées ainsi que différentes méthodes d'encodage des variables catégorielles (**WoE binning, One-Hot encoding**).

# Le Dataset (SBA 7(a) & 504)
Les données de départ (disponibles directement sur https://data.sba.gov/dataset/) contiennent plusieurs variables de différentes nature sur :
* **L'Emprunteur :** Localisation (`ProjectState`), Secteur d'activité (`NAICS`, `Industry`), Ancienneté.
* **La Structure du Prêt :** Montant (`GrossApproval`), Partie Garantie (`SBAGuaranteedApproval`), Durée (`TermInMonths`), Type (`RevolverStatus`).
* **Le Prêteur :** Qualité de la banque (`BankName`) et méthode d'octroi (`ProcessingMethod`).
* **Impact & Collatéral :** Emplois soutenus (`JobsSupported`) et présence de garanties (`CollateralInd`).
---

###### Importation des librairies

In [ ]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import plotly.express as px


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import P
ipeline
from category_encoders import TargetEncoder
from optbinning import OptimalBinning



from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from catboost import CatBoostClassifier, Pool, metrics


from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report

# 1. Récupération des données et premiers filtrage

In [ ]:
# 2000 - 2009
df1 = pd.read_csv('/Users/arthurdestribats/Downloads/foia-7a-fy2000-fy2009-asof-250930.csv')

# 2010 - 2019
df2 = pd.read_csv('/Users/arthurdestribats/Downloads/foia-7a-fy2010-fy2019-asof-250930.csv')

# 2020 - present
df3 = pd.read_csv('/Users/arthurdestribats/Downloads/foia-7a-fy2020-present-asof-250930.csv')

df = pd.concat([df1, df2,  df3])


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# GrossApproval correspond au montant total du prêt
# SBAGuaranteedApproval correspond au montant garantit SBA
pd.options.display.max_columns = None
df.head(3)

In [ ]:
df.isna().sum()/df.shape[0]

In [ ]:
for col in df.columns:
    if (df[col].isna().sum()/df.shape[0]) > 0.3:
        df = df.drop(col, axis= 1)
    else:
        pass

In [ ]:
df.isna().sum()/df.shape[0]

In [ ]:
df.loc[df["GrossChargeoffAmount"]>0]["LoanStatus"].value_counts()

In [ ]:
# target 
df['LoanStatus'].value_counts()

les modalités de la variable cible:
- **PIF** = PAID in full (donc remboursement total de l'entreprise et pas de défaut)
- **CHGOFF** = CHARGED OFF (donc l'emprunteur a arrêter de payer ==> Défaut)

ON a également:
- **COMMIT** = COMMITMENT / UNDISBURSED (à supprimer car le prêt n'a pas encore été donné)
- **CANCLD** = Cancelled (le contrat est annulé et le prêt n'a pas été approuvé donc à supprimer aussi de la base)

In [ ]:
df = df.loc[(df["LoanStatus"] == 'PIF') | (df['LoanStatus'] == 'CHGOFF')]

In [ ]:
df["LoanStatus"].value_counts(normalize=True)

In [ ]:
# Pour ne pas faire de data leakage et colonnes inutiles pour le moment
df = df.drop(columns={"GrossChargeoffAmount",
                               "AsOfDate"})

In [ ]:
# On binarise la cible
df["LoanStatus"] = np.where(df["LoanStatus"] == 'PIF', 0, 1)

# 2. ANALYSE DE DONNÉES

In [ ]:
pd.options.display.max_columns = None
df.head()

In [ ]:
# On garde seulement la dernière date d'obs d'un meme pret
df = df.drop_duplicates(subset=["BorrName", "BorrStreet", "BorrCity", "FirstDisbursementDate"], keep="last")
df = df.drop_duplicates()

In [ ]:
df["ApprovalDate"]= pd.to_datetime(df["ApprovalDate"], errors='coerce')
# Mise au format date
df["FirstDisbursementDate"] = pd.to_datetime(df["FirstDisbursementDate"])
df = df.set_index("ApprovalDate")

In [ ]:
df = df.drop(columns={'Program',
                      'BorrName',
                      'BorrStreet',
                      'BorrCity',
                      'LocationID',
                      'BankFDICNumber',
                      'BankZip',
                      'BankStreet',
                      'BankCity',
                      'NAICSDescription',
                      'CongressionalDistrict',
                      "BorrZip"
})

In [ ]:
print(df.shape)
df.columns

## 2.1. Valeurs manquantes

In [ ]:
# Pas de valeurs manquantes pour les variables numériques
col_numericals = ["GrossApproval", "SBAGuaranteedApproval", "TerminMonths", "JobsSupported"]
df[col_numericals].isna().sum()

In [ ]:
col_restants = [col for col in df.columns if col not in col_numericals]
df[col_restants].isna().sum()

In [ ]:
# On voit que lorsque Bankstate est manquant on a MISSINGMAINBANKID comme nom de banque (inconnu)
df.loc[df["BankState"].isna()]

# On va donc imputer ces valeurs dans une catégorie "unknown"
df["BankName"] = np.where(df["BankName"]=="MISSINGMAINBANKID", "unknown", df["BankName"])
df["BankState"] = np.where(df["BankState"].isna(), "unknown", df["BankState"])
print(f'Nombre de lignes imputées {df[df["BankState"] == "unknown"].shape[0]}')

In [ ]:
# Les lignes ou FirstDisbursementDate est manquant les autres variables sont renseignées
df.loc[df['FirstDisbursementDate'].isna()]
# Sachant que le premier versement arrive après un certain temps de la date d'approbation on peut prendre la médiane (dans tout les cas ca n'aura pas un grand impact car peu de valeurs manquantes)
approval_dates = df.index.to_series()
delai_median = (df['FirstDisbursementDate'] - approval_dates).median()
valeurs_remplacement = approval_dates + delai_median
df['FirstDisbursementDate'] = df['FirstDisbursementDate'].fillna(valeurs_remplacement)

In [ ]:
# Pour la variable CollateralInd on a deux possibilités (oui/non), pour povuoir garder la variable on fait l'hypothèse que NaN = No
df["CollateralInd"]=df["CollateralInd"].fillna(0)
df["CollateralInd"] = np.where(df['CollateralInd'] == "Y", 1, 0)
df.CollateralInd.value_counts()
# Seule possibilité est d'imputer par 'inconnu' pour NAICSCode
df["NAICSCode"] = df["NAICSCode"].fillna("inconnu")

## 2.2 Analyse de la variable numérique (cible dans le temps)

In [ ]:
defaut_par_an = df.groupby('ApprovalFY')['LoanStatus'].mean()

plt.figure(figsize=(12, 6))
sns.lineplot(data=defaut_par_an, marker='o', color='crimson', linewidth=2.5)

plt.title("Évolution du Taux de Défaut par Année d'Approbation")
plt.ylabel("Taux de Défaut (1 = 100%)")
plt.xlabel("Année Fiscale (FY)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.axvspan(2007, 2009, color='gray', alpha=0.2, label='Crise 2008')
plt.legend()
plt.show()

L’analyse de l’évolution temporelle du taux de défaut met en évidence plusieurs éléments importants :
- On observe un pic très marqué du nombre de défauts autour de 2008, ce qui correspond à la crise financière mondiale. Cette hausse est cohérente avec le contexte économique de l’époque.
- Un second pic, plus tardif, semble lié à l’impact différé de la crise du Covid-19, possiblement dû aux effets économiques retardés ou aux délais administratifs dans la déclaration des défauts.
- La période de 2008, bien qu’informative, peut potentiellement ajouter du bruit dans le modèle. Une solution pertinente serait d’introduire une variable indicatrice des périodes de crise, afin d’intégrer cette information de manière contrôlée.
- Enfin, la forte baisse observée sur la toute dernière période est probablement due à un manque de données (troncature du dataset) plutôt qu’à une amélioration réelle du risque.

## 2.3 Exploration de variables non numériques (intéressante)

### 2.3.1 Secteur d'activité

In [ ]:
# On remplace les valeurs manquantes ou vides par "unknown" pour éviter les NaN
df["BusinessType"] = np.where(df["BusinessType"] == " ", "unknown", df["BusinessType"])
# On affiche la distribution initiale
df.BusinessType.value_counts()

In [ ]:
# Les autres catégories ("unknown", " " et valeurs très rares) sont supprimées
df = df.loc[df["BusinessType"].isin(["CORPORATION", "INDIVIDUAL", "PARTNERSHIP"])]
# Vérification après nettoyage
df.BusinessType.value_counts()

####  PS : Exploration de la variable *NAICS Code*

Le NAICS code (North American Industry Classification System) permet d’identifier le **secteur d’activité** d’une entreprise.  
Les deux premiers chiffres correspondent au secteur macro-économique (exemples ci-dessous) :

| Code | Secteur | Exemple |
|------|---------|---------|
| 11 | Agriculture, forestry, fishing | 367,969 entreprises |
| 23 | Construction | 1,512,763 entreprises |
| 31–33 | Manufacturing | 660,462 entreprises |
| 44–45 | Retail trade | 1,478,617 entreprises |
| 52 | Finance & insurance | 771,419 entreprises |
| 62 | Health care & social assistance | 1,695,931 entreprises |

Ces catégories seront utiles pour :  
- **analyser le risque sectoriel**,  
- **créer des regroupements NAICS**,  
- **éventuellement intégrer des effets macro-économiques** dans le modèle.

In [ ]:
# Dictionnaire pour relier le cdes premiers chifres du code NAIC selon le secteur (voir image dessus)
naics_map = {
    '11': 'Agriculture', '21': 'Mining', '22': 'Utilities', '23': 'Construction',
    '31': 'Manufacturing', '32': 'Manufacturing', '33': 'Manufacturing',
    '42': 'Wholesale', '44': 'Retail', '45': 'Retail',
    '48': 'Transport', '49': 'Transport', '51': 'Information',
    '52': 'Finance', '53': 'Real Estate', '54': 'Prof. Services',
    '55': 'Management', '56': 'Admin/Waste', '61': 'Education',
    '62': 'Health Care', '71': 'Arts/Entertainment', '72': 'Accommodation/Food',
    '81': 'Other Services', '92': 'Public Admin', "in": "Inconnu"
}

# On prend les deux premiers chiffres
df['Industry'] = df['NAICSCode'].astype(str).str[:2]

# On fait le mapping du dico
df['IndustryName'] = df['Industry'].map(naics_map).fillna(df['Industry'])

# On calcule le taux de défaut par industrie
industry_risk = df.groupby('IndustryName')['LoanStatus'].mean().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(14, 8))
sns.barplot(x=industry_risk.values, 
            y=industry_risk.index, 
            hue=industry_risk.index)
plt.title("Taux de Défaut par Secteur d'Activité (Industry)")
plt.xlabel("Taux de Défaut")
plt.axvline(x=df['LoanStatus'].mean(), color='r', linestyle='--', label='Moyenne Globale')
plt.legend()
plt.show()

In [ ]:
# Taux de défaut selon le businesstype (individuel, corpo, partnership)
business_risk = df.groupby('BusinessType')['LoanStatus'].mean().sort_values(ascending=False)
plt.figure(figsize=(14, 8))
sns.barplot(x=business_risk.values, 
            y=business_risk.index, 
            hue=business_risk.index)
plt.title("Taux de Défaut par business type")
plt.xlabel("Taux de Défaut")
plt.axvline(x=df['LoanStatus'].mean(), color='r', linestyle='--', label='Moyenne Globale')
plt.legend()
plt.show()

Le graphique met en évidence des différences notables de taux de défaut selon le type d’entreprise :
- Individual présente le taux de défaut le plus élevé. Cela suggère que les prêts accordés aux entrepreneurs individuels comportent un risque intrinsèquement plus important, possiblement en raison d’une structure financière plus fragile ou d’une moindre diversification des revenus.
- Corporation affiche un taux de défaut intermédiaire, légèrement inférieur à la moyenne globale. Ce résultat indique un profil de risque plus maîtrisé, cohérent avec une structure juridique plus établie et un accès plus large aux ressources financières.
- Partnership est la catégorie présentant le taux de défaut le plus faible. Les partenariats semblent donc bénéficier d’une meilleure stabilité ou d’une gestion du risque plus efficace.

#### 2.3.2 variable géographique

In [ ]:
# les variables avec trop peu d'informations sont regroupées dans une catégorie other
freq = df['BorrState'].value_counts()
state_rares = freq[freq < 239].index
df.loc[df['BorrState'].isin(state_rares), 'BorrState'] = 'Other'

In [ ]:
freq = df['BankState'].value_counts()
state_rares = freq[freq < 400].index
df.loc[df['BankState'].isin(state_rares), 'BankState'] = 'Other'

bank_state_risk = df.groupby('BankState')['LoanStatus'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 15)) 
sns.barplot(x=bank_state_risk.values, 
            y=bank_state_risk.index, 
            hue=bank_state_risk.index, 
            legend=False)

plt.title("Taux de Défaut par État de la banque (du plus risqué au moins risqué) après regroupement des petits", fontsize=16)
plt.xlabel("Taux de Défaut", fontsize=12)
plt.ylabel("État", fontsize=12)

mean_val = df['LoanStatus'].mean()
plt.axvline(x=mean_val, color='black', linestyle='--', label=f'Moyenne Globale: {mean_val:.2%}')
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.legend()
plt.tight_layout() 
plt.show()

#### 2.3.3 Processus de prêt

In [ ]:
freq = df['ProcessingMethod'].value_counts()
rares = freq[freq < 1000].index
df.loc[df['ProcessingMethod'].isin(rares), 'ProcessingMethod'] = 'Other'
method_risk = df.groupby('ProcessingMethod')['LoanStatus'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 15)) 
sns.barplot(x=method_risk.values, 
            y=method_risk.index, 
            hue=method_risk.index, 
            legend=False)

plt.title("Taux de Défaut selon le process de prêt", fontsize=16)
plt.xlabel("Taux de Défaut", fontsize=12)
plt.ylabel("État", fontsize=12)

mean_val = df['LoanStatus'].mean()
plt.axvline(x=mean_val, color='black', linestyle='--', label=f'Moyenne Globale: {mean_val:.2%}')
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.legend()
plt.tight_layout() 
plt.show()

## 2.4 Conclusion de l’analyse exploratoire

Cette analyse exploratoire nous a permis de mieux comprendre la structure du dataset SBA et les principaux facteurs associés au risque de défaut.

Globalement, les données sont relativement propres, même si certaines variables présentaient des valeurs manquantes. L’étude temporelle met en évidence deux périodes très marquées : le pic de défauts autour de la crise financière de 2008, puis une hausse plus tardive liée aux effets du Covid. La chute observée sur la toute dernière année semble surtout due à un manque de données plutôt qu’à une amélioration réelle. Cela montre que le risque évolue fortement en fonction du contexte économique, et qu’il peut être pertinent d’intégrer une variable indiquant les périodes de crise.

Nous avons également observé que le risque n’est pas réparti de manière uniforme. Plusieurs dimensions influencent clairement le taux de défaut :
- le **type d’entreprise** (Individual étant de loin la catégorie la plus risquée),
- le **secteur d’activité** (certains secteurs comme le Retail, l’Information ou le Public Administration présentent un risque élevé),
- l’**État de la banque**, où des disparités importantes apparaissent,
- et le **processus de prêt SBA**, avec certains programmes plus risqués que d’autres.
